In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

In [57]:
frame = pd.read_csv('for_data_science_newline_fixed.csv', usecols = ["IN FILE","CLIP LENGTH","OFFSET","DURATION","MAX FREQ","MIN FREQ","SAMPLE RATE"])

In [58]:
# frame

In [59]:
# SET FEATURES TO ENGINEER
# MEDIAN ANNOTATION DURATIONS FOR FILE
# SUM OF ANNOTATION DURATIONS FOR FILE 
# NUMBER OF ANNOTATIONS
# MEDIAN DISTANCE BETWEEN ANNOTATIONS FOR FILE 


# BRAINSTORM:
# 

In [60]:
CLIP_LENGTHS = frame.groupby("IN FILE")[["CLIP LENGTH"]].max()

In [61]:
# CLIP_LENGTHS

In [62]:
SUM_DURATIONS = frame.groupby("IN FILE")[["DURATION"]].sum()

In [63]:
# SUM_DURATIONS

In [64]:
MEDIAN_DURATIONS = frame.groupby("IN FILE")[["DURATION"]].median()

In [65]:
MEDIAN_DURATIONS = MEDIAN_DURATIONS.rename(columns={'DURATION': 'MEDIAN DURATION'})
# MEDIAN_DURATIONS

In [66]:
NUMBER_ANN = frame.groupby("IN FILE")[["OFFSET"]].count()

In [67]:
NUMBER_ANN = NUMBER_ANN.rename(columns = {'OFFSET':'OFFSET COUNT'})
# NUMBER_ANN

In [68]:
# new column = (offset, offset+duration)
# new column containing end = offset+duration

# sort by start time, 
# groupby("FILE NAME")[["OFFSET"]].sort() 

MED_DIST_BW_ANNS = frame.copy(True)

MED_DIST_BW_ANNS = MED_DIST_BW_ANNS.sort_values(['IN FILE','OFFSET'])
MED_DIST_BW_ANNS['ENDSET'] = MED_DIST_BW_ANNS['OFFSET'] + MED_DIST_BW_ANNS['DURATION']

In [69]:
MED_DIST_BW_ANNS = MED_DIST_BW_ANNS[['IN FILE','CLIP LENGTH','OFFSET','ENDSET']]

In [70]:
# MED_DIST_BW_ANNS['SAME FILE'] = MED_DIST_BW_ANNS['IN FILE'].eq(MED_DIST_BW_ANNS['IN FILE'].shift())

In [71]:
# MED_DIST_BW_ANNS

In [72]:
MED_DIST_BW_ANNS['PREV END'] = MED_DIST_BW_ANNS.loc[MED_DIST_BW_ANNS['IN FILE'].shift(-1) == MED_DIST_BW_ANNS['IN FILE'], 'ENDSET']
MED_DIST_BW_ANNS['PREV END'] = MED_DIST_BW_ANNS['PREV END'].shift()
# MED_DIST_BW_ANNS

In [73]:
MED_DIST_BW_ANNS = MED_DIST_BW_ANNS.fillna(0)
MED_DIST_BW_ANNS['DIST BETWEEN ANN PREV'] = MED_DIST_BW_ANNS['OFFSET'] - MED_DIST_BW_ANNS['PREV END']
# MED_DIST_BW_ANNS

In [74]:
MED_DIST_BW_ANNS['OVERLAP'] = MED_DIST_BW_ANNS['DIST BETWEEN ANN PREV'].loc[MED_DIST_BW_ANNS['DIST BETWEEN ANN PREV'] < 0]

In [75]:
OVER_LAP = MED_DIST_BW_ANNS.groupby('IN FILE')[['OVERLAP']].count()
OVER_LAP = OVER_LAP.rename(columns = {'OVERLAP':'OVERLAP COUNT'})
# OVER_LAP

In [76]:
DIST_BETWEEN_ANN_PREV = MED_DIST_BW_ANNS.groupby('IN FILE')[['DIST BETWEEN ANN PREV']].median()
DIST_BETWEEN_ANN_PREV = DIST_BETWEEN_ANN_PREV.rename(columns={'DIST BETWEEN ANN PREV': 'MEDIAN DIST B/W ANNS'})
# DIST_BETWEEN_ANN_PREV

In [77]:
WHITE_SPACE = frame.copy(True)
DUR = WHITE_SPACE.groupby(['IN FILE'])[['DURATION']].sum()
DUR = DUR.rename(columns = {'DURATION':'SUM DURATION'})
LEN = WHITE_SPACE.groupby(['IN FILE'])[['CLIP LENGTH']].max()
WHITE_SPACE = LEN.join(DUR)
WHITE_SPACE['WHITE SPACE'] = WHITE_SPACE['CLIP LENGTH'] - WHITE_SPACE['SUM DURATION']
# WHITE_SPACE

In [78]:
WHITE_SPACE = WHITE_SPACE.join(OVER_LAP)
WHITE_SPACE = WHITE_SPACE.join(DIST_BETWEEN_ANN_PREV)
WHITE_SPACE = WHITE_SPACE.join(MEDIAN_DURATIONS)
WHITE_SPACE = WHITE_SPACE.join(NUMBER_ANN)
WHITE_SPACE.reset_index(inplace=True)
WHITE_SPACE

,IN FILE,CLIP LENGTH,SUM DURATION,WHITE SPACE,OVERLAP COUNT,MEDIAN DIST B/W ANNS,MEDIAN DURATION,OFFSET COUNT
0,Aburria-aburri-257423.wav,24.63350,8.7813,15.85220,0,2.36140,3.32050,3
1,Aburria-aburri-387212.wav,84.48000,9.4200,75.06000,0,26.76000,3.14000,3
2,Accipiter-bicolor-451839.wav,5.66857,4.6442,1.02437,3,0.00000,0.21550,20
3,Accipiter-collaris-260335.wav,45.37470,7.6708,37.70390,0,5.19555,1.03010,8
4,Accipiter-collaris-260336.wav,26.93220,4.8604,22.07180,1,1.03060,0.57010,8
...,...,...,...,...,...,...,...,...
2438,Zimmerius-gracilipes-258668.wav,12.19920,3.2975,8.90170,0,1.48075,0.81325,4
2439,Zimmerius-gracilipes-258669.wav,27.55920,10.8399,16.71930,0,0.68000,0.52000,21
2440,Zonotrichia-capensis-232609.wav,98.85600,31.5518,67.30420,0,3.73360,1.70010,17
2441,Zonotrichia-capensis-377483.wav,18.25960,7.7298,10.52980,0,3.01990,2.35990,3


In [79]:
def species(row):
    species = row['IN FILE']
    spec = species.split('-')
    # print(spec)
    # print(spec[:-1])
    spec_to = '-'.join(spec[:-2])
    return spec_to
WHITE_SPACE['Genus'] = WHITE_SPACE.apply (lambda row: species(row), axis=1)

In [80]:
WHITE_SPACE

,IN FILE,CLIP LENGTH,SUM DURATION,WHITE SPACE,OVERLAP COUNT,MEDIAN DIST B/W ANNS,MEDIAN DURATION,OFFSET COUNT,Genus
0,Aburria-aburri-257423.wav,24.63350,8.7813,15.85220,0,2.36140,3.32050,3,Aburria
1,Aburria-aburri-387212.wav,84.48000,9.4200,75.06000,0,26.76000,3.14000,3,Aburria
2,Accipiter-bicolor-451839.wav,5.66857,4.6442,1.02437,3,0.00000,0.21550,20,Accipiter
3,Accipiter-collaris-260335.wav,45.37470,7.6708,37.70390,0,5.19555,1.03010,8,Accipiter
4,Accipiter-collaris-260336.wav,26.93220,4.8604,22.07180,1,1.03060,0.57010,8,Accipiter
...,...,...,...,...,...,...,...,...,...
2438,Zimmerius-gracilipes-258668.wav,12.19920,3.2975,8.90170,0,1.48075,0.81325,4,Zimmerius
2439,Zimmerius-gracilipes-258669.wav,27.55920,10.8399,16.71930,0,0.68000,0.52000,21,Zimmerius
2440,Zonotrichia-capensis-232609.wav,98.85600,31.5518,67.30420,0,3.73360,1.70010,17,Zonotrichia
2441,Zonotrichia-capensis-377483.wav,18.25960,7.7298,10.52980,0,3.01990,2.35990,3,Zonotrichia


In [81]:
# WHITE_SPACE[['Genus']].value_counts().max
# WHITE_SPACE = WHITE_SPACE[(WHITE_SPACE['Genus'] == 'Piranga-leucoptera') | (WHITE_SPACE['Genus'] == 'Zonotrichia-capensis')]
# WHITE_SPACE

In [82]:
# if the following row is of the same species, take its off set and make a column to place hold it for the previous row.
# for the first annotation, take the distance from the start of the clip to start of annotation, for the last annotation 
# take the distance between the end of it and the end of the clip length

In [83]:
# use group by method or lambda function, 

In [84]:
#IQR On the Generated Features
WHITE_SPACE

,IN FILE,CLIP LENGTH,SUM DURATION,WHITE SPACE,OVERLAP COUNT,MEDIAN DIST B/W ANNS,MEDIAN DURATION,OFFSET COUNT,Genus
0,Aburria-aburri-257423.wav,24.63350,8.7813,15.85220,0,2.36140,3.32050,3,Aburria
1,Aburria-aburri-387212.wav,84.48000,9.4200,75.06000,0,26.76000,3.14000,3,Aburria
2,Accipiter-bicolor-451839.wav,5.66857,4.6442,1.02437,3,0.00000,0.21550,20,Accipiter
3,Accipiter-collaris-260335.wav,45.37470,7.6708,37.70390,0,5.19555,1.03010,8,Accipiter
4,Accipiter-collaris-260336.wav,26.93220,4.8604,22.07180,1,1.03060,0.57010,8,Accipiter
...,...,...,...,...,...,...,...,...,...
2438,Zimmerius-gracilipes-258668.wav,12.19920,3.2975,8.90170,0,1.48075,0.81325,4,Zimmerius
2439,Zimmerius-gracilipes-258669.wav,27.55920,10.8399,16.71930,0,0.68000,0.52000,21,Zimmerius
2440,Zonotrichia-capensis-232609.wav,98.85600,31.5518,67.30420,0,3.73360,1.70010,17,Zonotrichia
2441,Zonotrichia-capensis-377483.wav,18.25960,7.7298,10.52980,0,3.01990,2.35990,3,Zonotrichia


In [85]:
# how to take IQR, and know where a point lies in that range.
# Squeeze the values? for training, val, testing? plot has multiple rows.
# plotting in 3d.
# plotting in 3d using PCA

In [86]:
pca_data = WHITE_SPACE

In [87]:
# pca_data['IN FILE']=pd.Categorical(pca_data['IN FILE'])
# my_color=pca_data['IN FILE'].cat.codes

X = pca_data.drop(columns = ['IN FILE','Genus'])
X.head()

,CLIP LENGTH,SUM DURATION,WHITE SPACE,OVERLAP COUNT,MEDIAN DIST B/W ANNS,MEDIAN DURATION,OFFSET COUNT
0,24.63350,8.7813,15.85220,0,2.36140,3.3205,3
1,84.48000,9.4200,75.06000,0,26.76000,3.1400,3
2,5.66857,4.6442,1.02437,3,0.00000,0.2155,20
3,45.37470,7.6708,37.70390,0,5.19555,1.0301,8
4,26.93220,4.8604,22.07180,1,1.03060,0.5701,8


In [88]:
Y = np.array(pca_data['Genus'])
Y[:10]

array(['Aburria', 'Aburria', 'Accipiter', 'Accipiter', 'Accipiter',
       'Accipiter', 'Accipiter', 'Accipiter', 'Accipiter', 'Accipiter'],
      dtype=object)

In [89]:
# X

In [90]:
from sklearn.preprocessing import StandardScaler

In [91]:
scaler = StandardScaler(with_mean = False)
scaler.fit(X)
X_standardized = scaler.transform(X)
X_standard = pd.DataFrame(X_standardized, columns = X.columns)
X_standard

,CLIP LENGTH,SUM DURATION,WHITE SPACE,OVERLAP COUNT,MEDIAN DIST B/W ANNS,MEDIAN DURATION,OFFSET COUNT
0,0.625110,0.520108,0.504117,0.000000,0.459033,1.237198,0.177582
1,2.143800,0.557938,2.386987,0.000000,5.201882,1.169945,0.177582
2,0.143848,0.275072,0.032576,0.835953,0.000000,0.080294,1.183877
3,1.151447,0.454334,1.199024,0.000000,1.009964,0.383809,0.473551
4,0.683443,0.287877,0.701907,0.278651,0.200339,0.212416,0.473551
...,...,...,...,...,...,...,...
2438,0.309572,0.195308,0.283083,0.000000,0.287843,0.303012,0.236775
2439,0.699354,0.642037,0.531691,0.000000,0.132185,0.193749,1.243071
2440,2.508611,1.868783,2.140345,0.000000,0.725775,0.633447,1.006296
2441,0.463363,0.457829,0.334859,0.000000,0.587039,0.879284,0.177582


In [92]:
from sklearn.decomposition import PCA

In [93]:

pca = PCA(n_components = 3, random_state = 0)
pca.fit(X_standardized)
X_dim_reducted = pca.transform(X_standardized)
X_dim_reducted
result = pd.DataFrame(X_dim_reducted, columns = ['PCA1','PCA2','PCA3']) #,'PCA4','PCA5','PCA6'

In [94]:
import matplotlib

In [95]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# pca = PCA(n_components = 3)

# sns.scatterplot(x = 'PCA1', y = 'PCA2', hue = Y, palette = "Paired", data = result)

import plotly.express as px

# plt.figure(figsize=(16,10))
# ['Alphabet',
#  'Antique',
#  'Bold',
#  'D3',
#  'Dark2',
#  'Dark24',
#  'G10',......]

fig = px.scatter(result,x="PCA1", y="PCA2",color = Y) #legend = "Full", palette=sns.color_palette("hls", 947),,color_discrete_sequence= px.colors.qualitative.G10
fig.show() 

In [96]:
%matplotlib inline
# my_dpi=96
# plt.figure(figsize=(480/my_dpi, 480/my_dpi), dpi=my_dpi)

# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')

# ax.scatter(result['PCA1'], result['PCA2'], result['PCA3'], c=my_color, cmap="Set2_r", s=60)

# xAxisLine = ((min(result['PCA1']), max(result['PCA2'])), (0, 0), (0,0))
# ax.plot(xAxisLine[0], xAxisLine[1], xAxisLine[2], 'r')

# yAxisLine = ((0, 0), (min(result['PCA2']), max(result['PCA2'])), (0,0))
# ax.plot(yAxisLine[0], yAxisLine[1], yAxisLine[2], 'r')

# zAxisLine = ((0, 0), (0,0), (min(result['PCA3']), max(result['PCA3'])))
# ax.plot(zAxisLine[0], zAxisLine[1], zAxisLine[2], 'r')

# ax.set_xlabel("PC1")
# ax.set_ylabel("PC2")
# ax.set_zlabel("PC3")
# ax.set_title("PCA on Pyrenote Annotation Features")
# plt.show()

In [97]:
import plotly.express as px

In [98]:

fig = px.scatter_3d(result, x='PCA1', y='PCA2', z='PCA3',
                    color=Y)
fig.show()

In [99]:
import plotly.graph_objects as go
import numpy as np

# Helix equation
# t = np.linspace(0, 10, 50)
# x, y, z = result['PCA1'], result['PCA2'], result['PCA3']

# fig = go.Figure(data=[go.Scatter3d(text = Y, x=x, y=y, z=z,
#                                    mode='markers')])
# fig.show()

In [100]:
# import plotly.graph_objects as go
# import numpy as np

# # Helix equation
# # t = np.linspace(0, 20, 100)
# x, y, z = result['PCA1'], result['PCA2'], result['PCA3']#np.cos(t), np.sin(t), t

# fig = go.Figure(data=[go.Scatter3d(
#     text = Y,
#     x=x,
#     y=y,
#     z=z,
#     mode='markers',
#     marker=dict(
#         size=12,
#         color=z,                # set color to an array/list of desired values
#         colorscale='Viridis',   # choose a colorscale
#         opacity=0.8
#     )
# )])

# # tight layout
# fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
# fig.show()

In [101]:
# from sklearn.mixture import GaussianMixture

In [102]:
# gm = GaussianMixture(n_components = 3)
# gm.fit(X_standard)

In [103]:
# y_cluster_gmm = gm.predict(X_standard)
# y_cluster_gmm

In [104]:
# from sklearn.metrics.cluster import adjusted_rand_score

In [105]:
# score = adjusted_rand_score(Y, y_cluster_gmm)
# score

In [106]:
# from sklearn.cluster import KMeans

In [107]:
# kmeans = KMeans(n_clusters = 3)
# kmeans.fit(X_standard)

In [108]:
# y_cluster_kmeans = kmeans.predict(X_standard)
# y_cluster_kmeans

In [109]:
# score = adjusted_rand_score(Y, y_cluster_kmeans)
# score